In [20]:
#Python program to scrape details of all the mobile phones under Rs. 20,000 listed on Amazon.in
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests

no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/s?k=mobile+under+20000&'+str(pageNo)+'&crid=3MAJPKEBDMJ96&qid=1624261038&sprefix=mobile+under+200%2Caps%2C309&ref=sr_'+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'s-include-content-margin s-border-bottom s-latency-cf-section'}):
        #print(d)
        name = d.find('div', attrs={'class':'a-section aok-relative s-image-fixed-height'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        img_url = d.find('div', attrs={'class':'a-section aok-relative s-image-fixed-height'})
        m=img_url.find_all('img', src=True)
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('span', attrs={'class':'a-size-base'})
        price = d.find('span', attrs={'class':'a-price-whole'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if img_url is not None:
            #print(n[0]['src'])
            all1.append(n[0]['src'])
        else:
            all1.append("No Image")

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('Currently Unavailable')
        alls.append(all1)    
    return alls

results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Product Name','Image URL','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')
df.index += 1
df=df.style.set_properties(**{'text-align': 'left'})
df
